<h1>Feature extraction subject P03 </h1>

In [12]:
import numpy as np
import pandas as pd
pd.set_option('html',False)

from os import path

In [13]:
CHEST_X = path.relpath('data/P03_DATA_WINDOW/PO3_CHEST_X.csv')
CHEST_Y = path.relpath('data/P03_DATA_WINDOW/PO3_CHEST_Y.csv')
CHEST_Z = path.relpath('data/P03_DATA_WINDOW/PO3_CHEST_Z.csv')

THIGH_X = path.relpath('data/P03_DATA_WINDOW/PO3_THIGH_X.csv')
THIGH_Y = path.relpath('data/P03_DATA_WINDOW/PO3_THIGH_Y.csv')
THIGH_Z = path.relpath('data/P03_DATA_WINDOW/PO3_THIGH_Z.csv')

LABEL = path.relpath('data/P03_DATA_WINDOW/P03_LABEL_L.csv')

df_chest_x = pd.read_csv(CHEST_X, header=None, sep='\,')
df_chest_y = pd.read_csv(CHEST_Y, header=None, sep='\,')
df_chest_z = pd.read_csv(CHEST_Z, header=None, sep='\,')

df_thigh_x = pd.read_csv(THIGH_X, header=None, sep='\,')
df_thigh_y = pd.read_csv(THIGH_Y, header=None, sep='\,')
df_thigh_z = pd.read_csv(THIGH_Z, header=None, sep='\,')

df_label = pd.read_csv(LABEL, header=None, sep='\ ')


In [282]:
print len(df_chest_x), len(df_thigh_x)
print min(3,4)


36180 31115
3


<h2> Function to extract basic information for each window </h2>

In [283]:
'''
http://pandas.pydata.org/pandas-docs/stable/api.html#id5
Information about mean, median... 
'''

def extract(method, data_frame, start, size):
    res = []
    for i in range(start, start + size):
        
        if method == 'mean':
            res.append(data_frame.iloc[i].mean())
        elif method == 'median':
            res.append(data_frame.iloc[i].median())
        elif method == 'max':
            res.append(data_frame.iloc[i].max())
        elif method == 'min':
            res.append(data_frame.iloc[i].min())
        elif method == 'std':
            res.append(data_frame.iloc[i].std())
        else:

            res.append(data_frame.iloc[i])
        
    return pd.DataFrame(np.array(res))

In [284]:


def add_feature(feature_type,df_x,df_y,df_z,path,sensor):
    start=0
    size=min(len(df_chest_x), len(df_thigh_x))-1 
    
    

    FEATURES = path
    try:
        df_features = pd.read_csv(FEATURES, sep='\,')
        t = feature_type + '_x'
        if t not in list(df_features.columns.values):
        
            df_x_feature = extract(feature_type,df_x,start,size)
            df_x_feature.columns = [feature_type + '_x_' + sensor ]
            df_y_feature = extract(feature_type,df_y,start,size)
            df_y_feature.columns = [feature_type + '_y_' + sensor]
            df_z_feature= extract(feature_type,df_z,start,size)
            df_z_feature.columns = [feature_type + '_z_' + sensor ]
        
            df_features= pd.concat([df_features, df_x_feature,df_y_feature, df_z_feature],axis=1)
        
        
    except IOError:
        print 'file is empty'
        df_x_feature = extract(feature_type,df_x,start,size)
        df_x_feature.columns = [feature_type + '_x_' + sensor]
        df_y_feature = extract(feature_type,df_y,start,size)
        df_y_feature.columns = [feature_type + '_y_' + sensor]
        df_z_feature= extract(feature_type,df_z,start,size)
        df_z_feature.columns = [feature_type + '_z_' + sensor]
        
        df_features= pd.concat([df_x_feature,df_y_feature, df_z_feature],axis=1)



    
    df_features.to_csv(FEATURES, index=False)
    


In [285]:
path = '/home/astrid/Development/Prosjektoppgave/Notebook/data/FEATURES_P03.csv'
features = ['mean', 'min', 'max', 'median','std']

for f in features:
    add_feature(f, df_thigh_x, df_thigh_y, df_thigh_z, path, 'thigh')
    add_feature(f, df_chest_x, df_chest_y, df_chest_z, path, 'chest')

file is empty


<h2>Mean<h2>

In [286]:
start = 5000
size = 1000

<h3> Chest </h3>

In [287]:
df_chest_x_mean = extract('std',df_chest_x,start,size)
df_chest_y_mean = extract('std',df_chest_y,start,size)
df_chest_z_mean = extract('std',df_chest_z,start,size)
df_label_ = extract('',df_label,start,size)

df_chest_mean = pd.concat([df_chest_x_mean, df_chest_y_mean, df_chest_z_mean],axis=1,ignore_index=True)
df_chest_mean_with_label = pd.concat([df_chest_mean, df_label_],axis=1,ignore_index=True)


In [288]:
print df_chest_mean_with_label.tail()

            0         1         2   3
995  0.102030  0.101119  0.100589  12
996  0.115032  0.097500  0.067849  12
997  0.132985  0.140759  0.089236  12
998  0.116334  0.150775  0.075540  12
999  0.110227  0.137156  0.079348  12


<h3>Visualize mean values</h3>

In [289]:
import charts

In [290]:
options = dict(
    height=500, 
    title=dict(
        text='Mean chest'),
    chart=dict(zoomType='xy')
)

charts.plot(df_chest_mean_with_label, options=options, show='inline')

<h3>Thigh</h3>

In [291]:
df_thigh_x_mean = extract('std',df_thigh_x,start,size)
df_thigh_y_mean = extract('std',df_thigh_y,start,size)
df_thigh_z_mean = extract('std',df_thigh_z,start,size)
df_label_ = extract('',df_label,start,size)

df_thigh_mean = pd.concat([df_thigh_x_mean, df_thigh_y_mean, df_thigh_z_mean],axis=1,ignore_index=True)
df_thigh_mean_with_label = pd.concat([df_thigh_mean, df_label_],axis=1,ignore_index=True)

In [292]:
options = dict(
    height=500, 
    title=dict(
        text='Mean thigh'),
    chart=dict(zoomType='xy')
)

charts.plot(df_thigh_mean_with_label, options=options, show='inline')

In [293]:

df_chest_mean = df_chest_mean + 5
df_chest_thigh_mean = pd.concat([df_chest_mean,df_thigh_mean], axis=1,ignore_index=True)

options = dict(
    height=500, 
    title=dict(
        text='Mean chest'),
    chart=dict(zoomType='xy')
)

charts.plot(df_chest_thigh_mean, options=options, show='inline')